In [1]:
import pandas as pd
import numpy as np
import urllib2
import json
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_parced = pd.read_csv('train_parsed.csv',index=False)

In [2]:
train = pd.read_csv('champ1_train.csv')
test = pd.read_csv('champ1_test.csv')
new_test = pd.read_csv('new_test.csv')

In [5]:
new_test[new_test.city == 'no_city']

,_ID_,_CITY_,_ADRS_,_OBJ_TYPE_,_TTL_S_,_F1_S_,_F1_U_,_FC_S_,_FC_U_,_F0_S_,...,_DECOR_,_IS_COM_,_F1_H_,_DATE_,cityadddr,type,area,city,street,points
27,27,г. Санкт-Петербург,"ул. Типанова, 18, лит. А, пом. 16Н",помещение,"127,3","127,3",офисное,0,NaN,0,...,рабочее состояние,все,"2,95",05.05.2016,"г. Санкт-Петербург,ул. Типанова, 18, лит. А, п...",2,127.30,no_city,no_street,"0.0, 0.0"
45,45,г.Кондопога,"бульвар Юности д.18А,секция 3",помещение,"158,4",0,торговое,0,NaN,"158,4",...,рабочее состояние,все,нет данных,11.04.2016,"г.Кондопога,бульвар Юности д.18А,секция 3",1,158.40,no_city,no_street,"0.0, 0.0"
50,50,г.Петрозаводск,ул.Кирова д.7 пом.3,помещение,"42,1","42,1",офисное,0,NaN,0,...,евроремонт,все,нет данных,03.06.2016,"г.Петрозаводск,ул.Кирова д.7 пом.3",2,42.10,no_city,no_street,"0.0, 0.0"
51,51,г.Петрозаводск,"Октябрьский проспект д.63А, пом.1Н",помещение,"95,9","95,9",торговое,0,NaN,0,...,рабочее состояние,все,нет данных,23.06.2016,"г.Петрозаводск,Октябрьский проспект д.63А, пом.1Н",1,95.90,no_city,no_street,"0.0, 0.0"
54,54,г.Петрозаводск,"ул.Анохина д.35, помещение 1Н",помещение,"198,4","198,4",торговое,0,NaN,0,...,евроремонт,все,нет данных,04.07.2016,"г.Петрозаводск,ул.Анохина д.35, помещение 1Н",1,198.40,no_city,no_street,"0.0, 0.0"
55,55,г.Петрозаводск,пр.Лесной д.51 пом.2Н,помещение,"1688,4","1688,4",торговое,0,NaN,0,...,рабочее состояние,все,нет данных,04.07.2016,"г.Петрозаводск,пр.Лесной д.51 пом.2Н",1,1688.40,no_city,no_street,"0.0, 0.0"
57,57,г.Петрозаводск,ул.Володарского д.40 пом.42,помещение,"200,1","200,1",офисное,0,NaN,0,...,рабочее состояние,все,нет данных,19.09.2016,"г.Петрозаводск,ул.Володарского д.40 пом.42",2,200.10,no_city,no_street,"0.0, 0.0"
80,80,г. Санкт-Петербург,"улица Смолячкова, д.16, лит. С, пом. 1Н.",помещение,"82,6","82,6",офисное,0,NaN,0,...,рабочее состояние,все,"2,55",18.08.2016,"г. Санкт-Петербург, улица Смолячкова, д.16, ли...",2,82.60,no_city,no_street,"0.0, 0.0"
151,151,г. Санкт-Петербург,"коса Петровская, д. 1, корп. 1, лит. Р",помещение,"311,1","311,1",офисное,0,NaN,0,...,рабочее состояние,имеются частично,"3,73",12.04.2016,"г. Санкт-Петербург,коса Петровская, д. 1, корп...",2,311.10,no_city,no_street,"0.0, 0.0"
152,152,г. Санкт-Петербург,"коса Петровская, д. 1, корп. 1, лит. Р",помещение,"4,8","4,8",офисное,0,NaN,0,...,рабочее состояние,имеются частично,"3,73",12.04.2016,"г. Санкт-Петербург,коса Петровская, д. 1, корп...",2,4.80,no_city,no_street,"0.0, 0.0"


In [4]:
train = train[(train._PRICE_!='None')&(train._PRICE_!='Не указано')&(train._PRICE_!='Договорная')]
train = train[train._AREA_!='None']

In [5]:
train['price'] = [int(i.split('РУБ.')[0].replace(' ','')) for i in train._PRICE_]
train.price = train.price/1000
train.price = train.price.astype(int)

In [6]:
train['area'] = [float(i.split('м')[0].replace(',','.')) for i in train._AREA_]

In [43]:
def rmse(y, y_pred): 
    return mean_squared_error(y, y_pred)**0.5

In [44]:
scorer = make_scorer(rmse, greater_is_better=False)

In [7]:
def citystring(string):
    'make readable address string for geocoding'
    tstr = []
    for i in (string).split(','):
        for j in i.split(' '):
            if j!='':
                tstr.append(j)
    return '+'.join(tstr)

In [78]:
# geocoding by service: geocode-maps.yandex.ru

def geocode(lat, lon):
    'Geocode by coordinates:\
    returns county, city, street of first result'
    # make request (for additional info https://tech.yandex.ru/maps/doc/geocoder/desc/concepts/input_params-docpage/)
    req = urllib2.Request('https://geocode-maps.yandex.ru/1.x/?format=json&geocode={0},{1}'.format(lon,lat))
    # get data from response
    response = urllib2.urlopen(t)
    page = response.read()
    j = json.loads(page)
    
    try:
        city = j['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['description'].encode('utf-8')
    except:
        city = 'no_city'
    
    try:
        street = j['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['name'].encode('utf-8')
    except:
        street = 'no_street'
    
    # return city, county, country and street
    return city, street
    

def reverse_geocode(string):
    'Reverse geocode by address:\
    returns latitude and longitude of first result'
    # make request (for additional info https://tech.yandex.ru/maps/doc/geocoder/desc/concepts/input_params-docpage/)
    string = str(string)
    req_str = citystring(string)
    req = urllib2.Request('https://geocode-maps.yandex.ru/1.x/?format=json&geocode={0}'.format(req_str))
    # get data from response
    try:
        response = urllib2.urlopen(req)
        page = response.read()
        j = json.loads(page)

        try:
            city = j['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['description'].encode('utf-8')
        except:
            city = 'no_city'

        try:
            street = j['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['name'].encode('utf-8')
        except:
            street = 'no_street'

        try:
            point = j['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].encode('utf-8')
        except:
            point = '0.0, 0.0'

        return city, street, point
    except:
        city = 'no_city'
        street = 'no_street'
        point = '0.0, 0.0'
        return city, street, point

In [79]:
test['cityadddr'] = test._CITY_ + ',' + test._ADRS_

In [92]:
#cityes = []
#streets = []
#points = []

#for n,i in enumerate(list(test.cityadddr.values)):
#    if n%10 == 0:
#        print n, i
#    if n%500 == 0:
#        pd.DataFrame([cityes, streets, points]).transpose().to_csv(str(n)+'.csv',index=False)
#    city, street, point = reverse_geocode(i)
#    cityes.append(city)
#    streets.append(street)
#    points.append(point)

In [81]:
#test['city'] = cityes
#test['street'] = streets
#test['points'] = points

In [82]:
#test.to_csv('new_test.csv', index=False)

In [85]:
test['lat'] = [float(i.replace(',','').split(' ')[1]) for i in test.points]
test['lon'] = [float(i.replace(',','').split(' ')[0]) for i in test.points]

In [86]:
train_new = train[~train._LAT_.isnull()]

In [89]:
train_new = train_new[train_new._LAT_!='None']

In [93]:
train_new['lat'] = train_new._LAT_
train_new['lon'] = train_new._LON_

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
test.shape

(5404, 32)

In [15]:
test._F1_U_.value_counts()

торговое               2997
офисное                 931
складское               253
-0                      108
техническое              55
0                        17
торговое/складское        1
торговое, складское       1
торгово-офисное           1
Name: _F1_U_, dtype: int64

In [16]:
train._TYPE_.value_counts()

Другое          6482
Магазин         3567
Офис            2350
Производство    1736
Склад            702
Отель            665
None             239
Name: _TYPE_, dtype: int64

In [20]:
typ = []
for i in test._F1_U_.values:
    if i=='0' or i=='-0':
        typ.append(0)
    elif i=='торговое':
        typ.append(1)
    elif i=='офисное':
        typ.append(2)
    elif i=='складское':
        typ.append(3)
    else:
        typ.append(-1)

In [21]:
test['type'] = typ

In [23]:
typ = []
for i in train._TYPE_.values:
    if i=='Другое' or i=='None' or i=='Отель':
        typ.append(0)
    elif i=='Магазин':
        typ.append(1)
    elif i=='Офис':
        typ.append(2)
    elif i=='Склад':
        typ.append(3)
    else:
        typ.append(-1)

In [24]:
train['type'] = typ

In [37]:
area = []
for i in test._TTL_S_.values:
    try:
        area.append(float(str(i).replace(',','.')))
    except:
        area.append(0)
test['area'] = area

In [38]:
test.head()

,_ID_,_CITY_,_ADRS_,_OBJ_TYPE_,_TTL_S_,_F1_S_,_F1_U_,_FC_S_,_FC_U_,_F0_S_,...,_IS_WIN_,_IS_SEP_ENT_,_IS_VENT_,_DECOR_,_IS_COM_,_F1_H_,_DATE_,cityadddr,type,area
0,0,г. Санкт-Петербург,"1-я Красноармейская ул., д.8-10, лит.А",помещение,"184,7","184,7",торговое,0,NaN,0,...,да,да,центральная вентиляция,рабочее состояние,все,"2,7",19.04.2016,"г. Санкт-Петербург, 1-я Красноармейская ул., д...",1,184.7
1,1,г. Санкт-Петербург,"пр. Просвещения, д. 54, лит. А",помещение,"284,8","284,8",торговое,0,NaN,0,...,да,да,нет,рабочее состояние,все,"3/3,1/2,95",06.04.2016,"г. Санкт-Петербург,пр. Просвещения, д. 54, лит. А",1,284.8
2,2,г. Санкт-Петербург,"Кондратьевский, д. 51, корп. 1, лит. А",помещение,"59,6","59,6",торговое,0,NaN,0,...,да,да,нет,рабочее состояние,все,"2,75",11.04.2016,"г. Санкт-Петербург,Кондратьевский, д. 51, корп...",1,59.6
3,3,г. Санкт-Петербург,"ул. Лизы Чайкиной, д. 17, лит. А",помещение,"186,1",90,торговое,0,NaN,"96,1",...,да,да,нет,рабочее состояние,все,"1,65/3,65",11.04.2016,"г. Санкт-Петербург, ул. Лизы Чайкиной, д. 17, ...",1,186.1
4,4,г. Санкт-Петербург,"ул. Лизы Чайкиной, д. 18, лит. А",помещение,"73,3","73,3",торговое,0,NaN,0,...,нет,да,нет,рабочее состояние,все,4,11.04.2016,"г. Санкт-Петербург,ул. Лизы Чайкиной, д. 18, л...",1,73.3


In [26]:
train.head()

,_ID_﻿,_TYPE_,_DATE_,_ADDRESS_,_PRICE_,_AREA_,_METRO,_LAT_,_LON_,_DESC_,price,area,type
0,46595,Офис,10/09/2016,"г.Чебоксары, ул.Ярославская, 76, Ленинский, Че...",1 100 000 РУБ.,"16,1м²",NaN,56.13138,47.24982,"Офис находится в административном здании, кото...",1100,16.1,2
1,45544,Производство,15/08/2016,"ул Автодромная, 21, Троицк, Челябинская область",4 200 000 РУБ.,600м²,NaN,54.0818634,61.5924759,Продам производственное помещение в районе Тро...,4200,600.0,-1
2,17472,Другое,26/08/2016,"ул Дальневосточная, 151, Октябрьский, Иркутск,...",3 400 000 РУБ.,55м²,NaN,52.2538872,104.3219,"ЖК Ангарские паруса, отличное помещение, с отд...",3400,55.0,0
3,33269,Другое,30/08/2016,"ул Ленина, 268в, Азов, Ростовская область",15 000 000 РУБ.,280м²,NaN,47.0912743,39.4440079,Продается здание под коммерческую деятельность...,15000,280.0,0
4,21835,Офис,20/08/2016,"пр.Репина, Прикубанский, Краснодар, Краснодарс...",750 000 РУБ.,20м²,NaN,None,None,"цокольный этаж 16-ти эт. дома, квроремонт, спл...",750,20.0,2


In [27]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import cross_val_score

/home/ubuntu/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [101]:
score = []
for i in range(3,10,1):
    gb = GradientBoostingRegressor(n_estimators=1000, verbose=1, max_depth=i)
    score.append(cross_val_score(gb, train_new[['area','type','lat','lon']], train_new.price.values, cv=5, n_jobs=-1,scoring=scorer).mean())
    print(i, score)

      Iter       Train Loss   Remaining Time 
         1 81595087196.0891            2.00s
         2 79301868272.6525            1.97s
         3 77444338549.5295            1.96s
         4 72446058720.2573            1.94s
         5 68395577917.6932            1.93s
         6 65104573716.9471            1.93s
         7 63737645396.3929            1.93s
         8 61090605094.0995            1.93s
         9 60784943638.8999            1.92s
        10 60434687339.5618            1.91s
        20 52647363985.4643            1.79s
        30 49120731511.5455            1.77s
        40 46516712180.4392            1.71s
        50 44352816389.0939            1.67s
        60 39487319372.3573            1.65s
        70 38990660267.3961            1.63s
        80 37423388281.7229            1.60s
        90 36518281653.8675            1.57s
       100 35765628665.2118            1.55s
       200 28955216288.8676            1.37s
       300 27189170467.7601            1.19s
       40

KeyboardInterrupt: 

In [97]:
gb = GradientBoostingRegressor(n_estimators=1000, verbose=1, max_depth=5)
gb.fit(train[['area','type']], train.price.values)
vals = gb.predict(test[['area','type']])

      Iter       Train Loss   Remaining Time 
         1 71738821242.1220            5.19s
         2 69557020107.3563            5.04s
         3 66604769109.9794            5.03s
         4 64775536050.1113            4.98s
         5 63286601351.8485            4.98s
         6 61266843770.5451            4.94s
         7 60660054456.6573            4.91s
         8 59296732080.7324            4.90s
         9 57396963690.9347            4.88s
        10 56478615283.1545            4.89s
        20 46778027445.5468            4.68s
        30 38336001096.5436            4.38s
        40 36686946746.8303            4.17s
        50 33986610320.0744            4.05s
        60 32724260192.9077            3.95s
        70 31513925994.5720            3.85s
        80 29757063523.3402            3.76s
        90 28436029502.3606            3.68s
       100 27941112393.7608            3.63s
       200 22432464671.2370            3.17s
       300 21205720353.2291            2.75s
       40

In [98]:
df = pd.DataFrame([],columns=['_ID_','_PRICE_'])

In [99]:
df['_ID_'] = test._ID_.values
df['_PRICE_'] = vals

In [100]:
df.to_csv('baseline_2.csv',index=False)